In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

import tsvar

In [16]:
events = [
    torch.tensor([1.0, 3.0, 4.0, 5.0]),
    torch.tensor([2.0, 4.5, 6.0])
]

num_events = list(map(len, events))
num_events_tot = sum(num_events)
dim = len(events)

end_time = 6.2

mu = torch.tensor([0.2, 0.1])

beta = torch.tensor([1.0, 2.0])

alpha = torch.tensor([
    [0.1, 0.9],
    [0.4, 0.6]
])

coeffs = torch.cat((mu, beta, alpha.flatten()))

In [17]:
wmodel = tsvar.models.WoldModel()
wmodel.observe(events, end_time)

In [18]:
wmodel.delta_ikj[0]

tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [2.0000, 1.0000],
        [1.0000, 2.0000],
        [1.0000, 0.5000]])

In [19]:
wmodel.log_likelihood(coeffs)

0 tensor([0.2000, 0.2000, 0.3667, 0.3500, 0.4100])
tensor([0.1000, 0.4000])
tensor([[1., 2.]])
1 tensor([0.1000, 0.5500, 0.8333, 0.7214])
tensor([0.9000, 0.6000])
tensor([[1., 2.]])


tensor(-13.1327)

In [20]:
from math import log

lam_true = [
    np.array([
        0.2, 
        0.2, 
        0.2 + 0.1 / (1 + 2) + 0.4 / (2 + 1), 
        0.2 + 0.1 / (1 + 1) + 0.4 / (2 + 2)
    ]),
    np.array([
        0.1, 
        0.1 + 0.9 / (1 + 1), 
        0.1 + 0.9 / (1 + 0.5) + 0.6 / (2 + 2.5)
    ])
]

lam_end_true = [
    0.2 + 0.1 / (1 + 1) + 0.4 / (2 + 0.5),
    0.1 + 0.9 / (1 + 1) + 0.6 / (2 + 1.5)
]

ll_true = (
    np.log(lam_true[0]).sum() 
    - np.sum(lam_true[0] * np.hstack((events[0][0], np.diff(events[0]))))
    - lam_end_true[0] * (end_time - events[0][-1])
    
    + np.log(lam_true[1]).sum() 
    - np.sum(lam_true[1] * np.hstack((events[1][0], np.diff(events[1]))))
    - lam_end_true[1] * (end_time - events[1][-1])
)

ll_true

tensor(-13.1327)